In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('F1Stats_noWeather.csv')
df.sort_values(by=['raceId'], inplace=True)

def convert_time_to_decimal(time_str):
    # Extract minutes and seconds using regular expressions
    match = re.match(r'(\d+):(\d+)\.(\d+)', time_str)
    if match:
        minutes = int(match.group(1))
        seconds = int(match.group(2))
        return minutes + seconds / 100  # Convert to decimal format
    else:
        return pd.NA

# Convert time format to decimal format
df['pitDuration_decimal'] = pd.to_numeric(df['pitDuration'], errors='coerce')

# Convert remaining time format values to decimal format
df['pitDuration_decimal'].fillna(df['pitDuration'].apply(convert_time_to_decimal), inplace=True)

df.head(10)

,resultId,raceId,driverId,constructorId,grid,position,positionText,positionOrder,points,laps,...,constructorPoints,constructorPosition,constructorPositionText,constructorWins,stop,lap,pitTime,pitDuration,milliseconds_y,pitDuration_decimal
0,20779,841,20,9,1,1,1,1,25.0,58,...,35.0,1,1,1,1,14,17:25:17,22.603,22603,22.603
23,20786,841,67,5,10,8,8,8,4.0,57,...,4.0,5,5,0,1,15,17:27:34,25.342,25342,25.342
24,20779,841,20,9,1,1,1,1,25.0,58,...,35.0,1,1,1,2,36,17:59:17,24.036,24036,24.036
25,20783,841,17,9,3,5,5,5,10.0,58,...,35.0,1,1,1,1,11,17:20:48,23.426,23426,23.426
26,20783,841,17,9,3,5,5,5,10.0,58,...,35.0,1,1,1,2,26,17:44:29,22.520,22520,22.520
27,20780,841,1,1,2,2,2,2,18.0,58,...,26.0,2,2,0,1,16,17:28:24,23.227,23227,23.227
28,20780,841,1,1,2,2,2,2,18.0,58,...,26.0,2,2,0,2,36,17:59:29,23.199,23199,23.199
29,20784,841,18,1,4,6,6,6,8.0,58,...,26.0,2,2,0,1,17,17:30:24,16.867,16867,16.867
30,20784,841,18,1,4,6,6,6,8.0,58,...,26.0,2,2,0,2,19,17:33:53,23.303,23303,23.303
22,20785,841,13,6,8,7,7,7,6.0,58,...,18.0,3,3,0,3,48,18:18:54,24.095,24095,24.095


In [3]:
print(len(df))

9631


In [4]:
df.dtypes

resultId                     int64
raceId                       int64
driverId                     int64
constructorId                int64
grid                         int64
position                    object
positionText                object
positionOrder                int64
points                     float64
laps                         int64
time                        object
milliseconds_x              object
fastestLap                  object
rank                         int64
fastestLapTime              object
fastestLapSpeed             object
statusId                     int64
year                         int64
round                        int64
circuitId                    int64
raceName                    object
date                        object
raceTime                    object
fp1_date                    object
fp1_time                    object
fp2_date                    object
fp2_time                    object
fp3_date                    object
fp3_time            

In [5]:
print(df.columns)
print(len(df.columns))

Index(['resultId', 'raceId', 'driverId', 'constructorId', 'grid', 'position',
       'positionText', 'positionOrder', 'points', 'laps', 'time',
       'milliseconds_x', 'fastestLap', 'rank', 'fastestLapTime',
       'fastestLapSpeed', 'statusId', 'year', 'round', 'circuitId', 'raceName',
       'date', 'raceTime', 'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time',
       'fp3_date', 'fp3_time', 'quali_date', 'quali_time', 'sprint_date',
       'sprint_time', 'name', 'location', 'country', 'lat', 'lng', 'alt',
       'wins', 'code', 'forename', 'surname', 'nationality', 'constructorName',
       'constructorPoints', 'constructorPosition', 'constructorPositionText',
       'constructorWins', 'stop', 'lap', 'pitTime', 'pitDuration',
       'milliseconds_y', 'pitDuration_decimal'],
      dtype='object')
55


In [6]:
df.drop(['milliseconds_y', 'stop', 'lap', 'pitDuration', 'milliseconds_x'], inplace=True, axis=1)
df.rename(columns={'pitDuration_decimal': 'pitDuration'}, inplace=True)

In [7]:
aggregated_df = df.copy()
aggregated_df = aggregated_df.groupby(['raceId', 'driverId']).agg({'pitTime': 'count', 'pitDuration': 'mean', **{col: 'first' for col in aggregated_df.columns if col not in ['raceId', 'driverId', 'pitTime', 'pitDuration']}}).reset_index()

In [8]:
aggregated_df.rename(columns={'pitTime': 'pitStops'}, inplace=True)
aggregated_df

,raceId,driverId,pitStops,pitDuration,resultId,constructorId,grid,position,positionText,positionOrder,...,wins,code,forename,surname,nationality,constructorName,constructorPoints,constructorPosition,constructorPositionText,constructorWins
0,841,1,2,23.2130,20780,1,2,2,2,2,...,0,HAM,Lewis,Hamilton,British,McLaren,26.0,2,2,0
1,841,2,2,24.0460,20790,4,18,12,12,12,...,0,HEI,Nick,Heidfeld,German,Renault,15.0,4,4,0
2,841,3,1,23.7160,20795,131,7,\N,R,17,...,0,ROS,Nico,Rosberg,German,Mercedes,0.0,10,10,0
3,841,4,3,24.0550,20782,6,5,4,4,4,...,0,ALO,Fernando,Alonso,Spanish,Ferrari,18.0,3,3,0
4,841,5,1,24.8650,20796,205,19,\N,R,18,...,0,KOV,Heikki,Kovalainen,Finnish,Lotus,0.0,7,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4694,1096,848,2,21.7955,25838,3,19,13,13,13,...,0,ALB,Alexander,Albon,Thai,Williams,8.0,10,10,0
4695,1096,849,2,23.7985,25844,3,20,19,19,19,...,0,LAT,Nicholas,Latifi,Canadian,Williams,8.0,10,10,0
4696,1096,852,2,21.9265,25836,213,11,11,11,11,...,0,TSU,Yuki,Tsunoda,Japanese,AlphaTauri,35.0,9,9,0
4697,1096,854,2,22.9030,25841,210,12,16,16,16,...,0,MSC,Mick,Schumacher,German,Haas F1 Team,37.0,8,8,0


In [9]:
count_n_values = (df == '\\N').sum()
columns_with_n_values = count_n_values[count_n_values > 0].index.tolist()
columns_with_n_values

['position',
 'time',
 'fastestLap',
 'fastestLapTime',
 'fastestLapSpeed',
 'fp1_date',
 'fp1_time',
 'fp2_date',
 'fp2_time',
 'fp3_date',
 'fp3_time',
 'quali_date',
 'quali_time',
 'sprint_date',
 'sprint_time',
 'alt']

In [10]:
for column in aggregated_df.columns:
    aggregated_df[column] = aggregated_df[column].replace('\\N', 0)

## Create DNF columns

In [11]:
df[(df['raceId'] == 841) & (df['driverId'] == 815)]['statusId']

Series([], Name: statusId, dtype: int64)

In [12]:
for i in range(1, 142):
    print(i)
    print(df[df['statusId'] == i]['positionText'].unique())
    # print(df[df['statusId'] == i][['raceName', 'date', 'positionText', 'forename']].head(5))

1
['1' '5' '2' '6' '7' '3' '4' '8' '10' '9' '12' '11' '13' '14' '19' '18'
 '17' '15' '16']
2
['D']
3
['R' '17' '18' '21' '13' '19' '20']
4
['R' '16' '19' '20' '12' '11' '18' '14' '17']
5
['R' '19' '11' '18' '15']
6
['R' '18' '17' '15']
7
['R' '17']
8
['R' '17']
9
['R' '18']
10
['R']
11
['8' '12' '11' '9' '10' '13' '15' '14' '17' '18' '16' '19' '7' '6' '5'
 '20' '21' '22' '4']
12
['13' '16' '23' '22' '21' '20' '19' '18' '17' '14' '11' '12' '15' '9' '10'
 '7' '6' '8']
13
['21' '19' '15' '16' '23' '24' '18' '17' '20' '22' '14' '12' '13']
14
['14' '20' '17' '18' '19' '16' '15']
15
['22' '21' '20' '19' '15']
16
['18' '16']
17
[]
18
['N' '17']
19
['N']
20
['R']
21
['R']
22
['R' '21' '16' '15' '19' '12']
23
['21' 'R' '20' '17' '19' '15' '18']
24
['R']
25
['R']
26
['R' '19']
27
['R' '12' '13']
28
[]
29
['R' '17']
30
['R']
31
['R' '18' '22' '19' '16' '20' '17']
32
['R' '18' '19']
33
['R' '16']
34
['15' 'R' '14']
35
[]
36
['R' '21' '20' '16']
37
['R']
38
['R']
39
['R']
40
['R' '13']
41
[]
42
['R

In [13]:
dnf_reasons_driver = [3,4,20,29,31,41,68,73,81,82,97,100,104,107,111,130,135,136,139] # based on statusId
dnf_reasons_constructor = [5,6,7,8,9,10,21,22,23,24,25,26,27,28,30,32,33,34,36,37,38,39,40,42,43,44,48,49,51,56,129,65,66,67,69,70,71,72,74,75,76,77,78,79,80,83,84,85,86,87,90,91,92,93,94,95,98,99,103,105,106,108,109,110,121,126,131,132,140,141] # based on statusId

In [14]:
aggregated_df['driver_dnf'] = aggregated_df['statusId'].apply(lambda x: 1 if x in dnf_reasons_driver else 0)
aggregated_df['constructor_dnf'] = aggregated_df['statusId'].apply(lambda x: 1 if x in dnf_reasons_constructor else 0)

In [15]:
print(aggregated_df['driver_dnf'].sum())
print(aggregated_df['constructor_dnf'].sum())

239
313


### Get the reliability and driver confidence

In [16]:
def get_reliability_driver_const(data, columnId, column):
  # Solo vamos a calcular 2 año atras la informacion de cuantas carrreras entro y cuantas veces tuvo dnf, en vez de calcular de todo el historial de las carreras
  data_temp_dnf = data.copy()
  data_temp_dnf['date'] = pd.to_datetime(data_temp_dnf['date'])
  data_temp_dnf['index_column'] = data_temp_dnf.index
  data_temp_dnf.set_index('date', inplace=True)
  data_temp_dnf.sort_index(inplace=True)


  window_size = '730D' # 2 years

  # data_temp_dnf['rolling_sum_1'] = data_temp_dnf.groupby('driver')['driver_dnf'].apply(lambda x: x.shift().rolling(window_size).sum())
  # data_temp_dnf['rolling_sum_2'] = data_temp_dnf.groupby('driver')['driver_dnf'].apply(lambda x: x.shift().rolling(window_size).count())

  all_columns = data[columnId].unique()

  for column_row in all_columns:
      data_temp_dnf.loc[data_temp_dnf[columnId] == column_row, 'total_dnf'] = data_temp_dnf[data_temp_dnf[columnId] == column_row].shift().rolling(window_size)[f'{column}_dnf'].sum()
      data_temp_dnf.loc[data_temp_dnf[columnId] == column_row, 'total_races'] = data_temp_dnf[data_temp_dnf[columnId] == column_row].shift().rolling(window_size)[f'{column}_dnf'].count()


  data_temp_dnf['dnf_ratio'] = 1 - (data_temp_dnf['total_dnf'] / data_temp_dnf['total_races'])

  data_temp_dnf.reset_index(inplace=True)
  data_temp_dnf.set_index('index_column', inplace=True)
  data_temp_dnf.sort_index(inplace=True)
  data_temp_dnf.reset_index(inplace=True)

  return data_temp_dnf['dnf_ratio']

In [17]:
aggregated_df['driver_confidence'] = get_reliability_driver_const(aggregated_df, 'driverId', 'driver')
aggregated_df['driver_confidence'] = aggregated_df['driver_confidence'].fillna(0)
aggregated_df['constructor_reliability'] = get_reliability_driver_const(aggregated_df, 'constructorId', 'constructor')
aggregated_df['constructor_reliability'] = aggregated_df['constructor_reliability'].fillna(0)

### Fix qualification position 0

We give them the last position for grid position

Change the values manually investigating on the internet

In [18]:
aggregated_df[aggregated_df['grid'] == 0][['raceName', 'date', 'forename', 'surname', 'positionOrder', 'grid', 'raceId', 'driverId']]

,raceName,date,forename,surname,positionOrder,grid,raceId,driverId
1758,Monaco Grand Prix,2015-05-24,Carlos,Sainz,10,0,931,832
2115,Monaco Grand Prix,2016-05-29,Max,Verstappen,18,0,953,830
2116,Monaco Grand Prix,2016-05-29,Felipe,Nasr,17,0,953,831
2166,Austrian Grand Prix,2016-07-03,Felipe,Massa,20,0,956,13
3228,Chinese Grand Prix,2019-04-14,Alexander,Albon,10,0,1012,848
3230,Azerbaijan Grand Prix,2019-04-28,Kimi,Räikkönen,10,0,1013,8
3231,Azerbaijan Grand Prix,2019-04-28,Robert,Kubica,16,0,1013,9
3253,Spanish Grand Prix,2019-05-12,Nico,Hülkenberg,13,0,1014,807
3297,Canadian Grand Prix,2019-06-09,Kevin,Magnussen,17,0,1016,825
3345,Austrian Grand Prix,2019-06-30,George,Russell,18,0,1018,847


In [19]:
aggregated_df.loc[(aggregated_df['raceId'] == 931) & (aggregated_df['driverId'] == 832), 'grid'] = 20
aggregated_df.loc[(aggregated_df['raceId'] == 953) & (aggregated_df['driverId'] == 830), 'grid'] = 22
aggregated_df.loc[(aggregated_df['raceId'] == 953) & (aggregated_df['driverId'] == 831), 'grid'] = 21
aggregated_df.loc[(aggregated_df['raceId'] == 956) & (aggregated_df['driverId'] == 13), 'grid'] = 22
aggregated_df.loc[(aggregated_df['raceId'] == 1012) & (aggregated_df['driverId'] == 848), 'grid'] = 22
aggregated_df.loc[(aggregated_df['raceId'] == 1013) & (aggregated_df['driverId'] == 8), 'grid'] = 19
aggregated_df.loc[(aggregated_df['raceId'] == 1013) & (aggregated_df['driverId'] == 9), 'grid'] = 18
aggregated_df.loc[(aggregated_df['raceId'] == 1014) & (aggregated_df['driverId'] == 807), 'grid'] = 20
aggregated_df.loc[(aggregated_df['raceId'] == 1016) & (aggregated_df['driverId'] == 825), 'grid'] = 20
aggregated_df.loc[(aggregated_df['raceId'] == 1016) & (aggregated_df['driverId'] == 825), 'grid'] = 20
aggregated_df.loc[(aggregated_df['raceId'] == 1018) & (aggregated_df['driverId'] == 847), 'grid'] = 20
aggregated_df.loc[(aggregated_df['raceId'] == 1022) & (aggregated_df['driverId'] == 9), 'grid'] = 20
aggregated_df.loc[(aggregated_df['raceId'] == 1023) & (aggregated_df['driverId'] == 8), 'grid'] = 20
aggregated_df.loc[(aggregated_df['raceId'] == 1025) & (aggregated_df['driverId'] == 848), 'grid'] = 20
aggregated_df.loc[(aggregated_df['raceId'] == 1026) & (aggregated_df['driverId'] == 9), 'grid'] = 20
aggregated_df.loc[(aggregated_df['raceId'] == 1028) & (aggregated_df['driverId'] == 815), 'grid'] = 20

aggregated_df.loc[(aggregated_df['raceId'] == 1032) & (aggregated_df['driverId'] == 154), 'grid'] = 20
aggregated_df.loc[(aggregated_df['raceId'] == 1033) & (aggregated_df['driverId'] == 154), 'grid'] = 18
aggregated_df.loc[(aggregated_df['raceId'] == 1033) & (aggregated_df['driverId'] == 825), 'grid'] = 16
aggregated_df.loc[(aggregated_df['raceId'] == 1044) & (aggregated_df['driverId'] == 847), 'grid'] = 20
aggregated_df.loc[(aggregated_df['raceId'] == 1044) & (aggregated_df['driverId'] == 849), 'grid'] = 18
aggregated_df.loc[(aggregated_df['raceId'] == 1052) & (aggregated_df['driverId'] == 815), 'grid'] = 11
aggregated_df.loc[(aggregated_df['raceId'] == 1053) & (aggregated_df['driverId'] == 20), 'grid'] = 13
aggregated_df.loc[(aggregated_df['raceId'] == 1053) & (aggregated_df['driverId'] == 20), 'grid'] = 13
aggregated_df.loc[(aggregated_df['raceId'] == 1059) & (aggregated_df['driverId'] == 852), 'grid'] = 20
aggregated_df.loc[(aggregated_df['raceId'] == 1061) & (aggregated_df['driverId'] == 815), 'grid'] = 20
aggregated_df.loc[(aggregated_df['raceId'] == 1062) & (aggregated_df['driverId'] == 841), 'grid'] = 14
aggregated_df.loc[(aggregated_df['raceId'] == 1064) & (aggregated_df['driverId'] == 815), 'grid'] = 20
aggregated_df.loc[(aggregated_df['raceId'] == 1064) & (aggregated_df['driverId'] == 849), 'grid'] = 19
aggregated_df.loc[(aggregated_df['raceId'] == 1071) & (aggregated_df['driverId'] == 8), 'grid'] = 13
aggregated_df.loc[(aggregated_df['raceId'] == 1071) & (aggregated_df['driverId'] == 8), 'grid'] = 13
aggregated_df.loc[(aggregated_df['raceId'] == 1077) & (aggregated_df['driverId'] == 855), 'grid'] = 20
aggregated_df.loc[(aggregated_df['raceId'] == 1078) & (aggregated_df['driverId'] == 20), 'grid'] = 20
aggregated_df.loc[(aggregated_df['raceId'] == 1078) & (aggregated_df['driverId'] == 840), 'grid'] = 19

aggregated_df.loc[(aggregated_df['raceId'] == 1084) & (aggregated_df['driverId'] == 822), 'grid'] = 20
aggregated_df.loc[(aggregated_df['raceId'] == 1086) & (aggregated_df['driverId'] == 842), 'grid'] = 20
aggregated_df.loc[(aggregated_df['raceId'] == 1087) & (aggregated_df['driverId'] == 842), 'grid'] = 8
aggregated_df.loc[(aggregated_df['raceId'] == 1087) & (aggregated_df['driverId'] == 852), 'grid'] = 20
aggregated_df.loc[(aggregated_df['raceId'] == 1091) & (aggregated_df['driverId'] == 847), 'grid'] = 20
aggregated_df.loc[(aggregated_df['raceId'] == 1093) & (aggregated_df['driverId'] == 839), 'grid'] = 20
aggregated_df.loc[(aggregated_df['raceId'] == 1095) & (aggregated_df['driverId'] == 852), 'grid'] = 20

aggregated_df

,raceId,driverId,pitStops,pitDuration,resultId,constructorId,grid,position,positionText,positionOrder,...,nationality,constructorName,constructorPoints,constructorPosition,constructorPositionText,constructorWins,driver_dnf,constructor_dnf,driver_confidence,constructor_reliability
0,841,1,2,23.2130,20780,1,2,2,2,2,...,British,McLaren,26.0,2,2,0,0,0,0.000000,0.000000
1,841,2,2,24.0460,20790,4,18,12,12,12,...,German,Renault,15.0,4,4,0,0,0,0.000000,1.000000
2,841,3,1,23.7160,20795,131,7,0,R,17,...,German,Mercedes,0.0,10,10,0,1,0,0.000000,1.000000
3,841,4,3,24.0550,20782,6,5,4,4,4,...,Spanish,Ferrari,18.0,3,3,0,0,0,0.000000,1.000000
4,841,5,1,24.8650,20796,205,19,0,R,18,...,Finnish,Lotus,0.0,7,7,0,0,0,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4694,1096,848,2,21.7955,25838,3,19,13,13,13,...,Thai,Williams,8.0,10,10,0,0,0,0.952381,0.928571
4695,1096,849,2,23.7985,25844,3,20,19,19,19,...,Canadian,Williams,8.0,10,10,0,1,0,0.860465,0.929412
4696,1096,852,2,21.9265,25836,213,11,11,11,11,...,Japanese,AlphaTauri,35.0,9,9,0,0,0,0.921053,0.930233
4697,1096,854,2,22.9030,25841,210,12,16,16,16,...,German,Haas F1 Team,37.0,8,8,0,0,0,0.973684,0.939759


In [20]:
# unique_races = aggregated_df['raceId'].unique()

# for i in unique_races:
#     racers = len(aggregated_df[aggregated_df['raceId'] == i])
#     quali_positions = aggregated_df[aggregated_df['raceId'] == i]['grid'].unique().max()
#     amount_of_0 = len(aggregated_df[(aggregated_df['raceId'] == i) & (aggregated_df['grid'] == 0)])

#     condition = (aggregated_df['raceId'] == i) & (aggregated_df['grid'] == 0)
#     row_index = aggregated_df.loc[condition]
                      
#     for j in range(amount_of_0):
#         row_index = aggregated_df.loc[condition].index[j]
#         aggregated_df.at[row_index, 'grid'] = quali_positions + j + 1

In [21]:
len(aggregated_df[aggregated_df['grid'] == 0])

0

## Create final df

In [22]:
aggregated_df['driver'] = aggregated_df['forename'].str.cat(aggregated_df['surname'], sep=" ")

In [23]:
final_df = aggregated_df[['year', 'date', 'raceName', 'raceId', 'circuitId', 'driverId', 'constructorId', 'driver', 'constructorName', 'positionOrder', 'grid', 'wins', 'points', 'constructorPoints', 'constructorWins', 'nationality', 'statusId', 'pitStops', 'pitDuration', 'raceName', 'constructor_dnf', 'driver_dnf', 'constructor_reliability', 'driver_confidence']]
final_df

,year,date,raceName,raceId,circuitId,driverId,constructorId,driver,constructorName,positionOrder,...,constructorWins,nationality,statusId,pitStops,pitDuration,raceName,constructor_dnf,driver_dnf,constructor_reliability,driver_confidence
0,2011,2011-03-27,Australian Grand Prix,841,1,1,1,Lewis Hamilton,McLaren,2,...,0,British,1,2,23.2130,Australian Grand Prix,0,0,0.000000,0.000000
1,2011,2011-03-27,Australian Grand Prix,841,1,2,4,Nick Heidfeld,Renault,12,...,0,German,11,2,24.0460,Australian Grand Prix,0,0,1.000000,0.000000
2,2011,2011-03-27,Australian Grand Prix,841,1,3,131,Nico Rosberg,Mercedes,17,...,0,German,4,1,23.7160,Australian Grand Prix,0,1,1.000000,0.000000
3,2011,2011-03-27,Australian Grand Prix,841,1,4,6,Fernando Alonso,Ferrari,4,...,0,Spanish,1,3,24.0550,Australian Grand Prix,0,0,1.000000,0.000000
4,2011,2011-03-27,Australian Grand Prix,841,1,5,205,Heikki Kovalainen,Lotus,18,...,0,Finnish,47,1,24.8650,Australian Grand Prix,0,0,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4694,2022,2022-11-20,Abu Dhabi Grand Prix,1096,24,848,3,Alexander Albon,Williams,13,...,0,Thai,11,2,21.7955,Abu Dhabi Grand Prix,0,0,0.928571,0.952381
4695,2022,2022-11-20,Abu Dhabi Grand Prix,1096,24,849,3,Nicholas Latifi,Williams,19,...,0,Canadian,130,2,23.7985,Abu Dhabi Grand Prix,0,1,0.929412,0.860465
4696,2022,2022-11-20,Abu Dhabi Grand Prix,1096,24,852,213,Yuki Tsunoda,AlphaTauri,11,...,0,Japanese,1,2,21.9265,Abu Dhabi Grand Prix,0,0,0.930233,0.921053
4697,2022,2022-11-20,Abu Dhabi Grand Prix,1096,24,854,210,Mick Schumacher,Haas F1 Team,16,...,0,German,11,2,22.9030,Abu Dhabi Grand Prix,0,0,0.939759,0.973684


In [24]:
final_df = final_df.rename(columns={'positionOrder': 'position', 'grid': 'quali_pos', 'nationality': 'driver_home', 'raceName': 'GP_name', 'constructorName': 'constructor'})
final_df

,year,date,GP_name,raceId,circuitId,driverId,constructorId,driver,constructor,position,...,constructorWins,driver_home,statusId,pitStops,pitDuration,GP_name,constructor_dnf,driver_dnf,constructor_reliability,driver_confidence
0,2011,2011-03-27,Australian Grand Prix,841,1,1,1,Lewis Hamilton,McLaren,2,...,0,British,1,2,23.2130,Australian Grand Prix,0,0,0.000000,0.000000
1,2011,2011-03-27,Australian Grand Prix,841,1,2,4,Nick Heidfeld,Renault,12,...,0,German,11,2,24.0460,Australian Grand Prix,0,0,1.000000,0.000000
2,2011,2011-03-27,Australian Grand Prix,841,1,3,131,Nico Rosberg,Mercedes,17,...,0,German,4,1,23.7160,Australian Grand Prix,0,1,1.000000,0.000000
3,2011,2011-03-27,Australian Grand Prix,841,1,4,6,Fernando Alonso,Ferrari,4,...,0,Spanish,1,3,24.0550,Australian Grand Prix,0,0,1.000000,0.000000
4,2011,2011-03-27,Australian Grand Prix,841,1,5,205,Heikki Kovalainen,Lotus,18,...,0,Finnish,47,1,24.8650,Australian Grand Prix,0,0,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4694,2022,2022-11-20,Abu Dhabi Grand Prix,1096,24,848,3,Alexander Albon,Williams,13,...,0,Thai,11,2,21.7955,Abu Dhabi Grand Prix,0,0,0.928571,0.952381
4695,2022,2022-11-20,Abu Dhabi Grand Prix,1096,24,849,3,Nicholas Latifi,Williams,19,...,0,Canadian,130,2,23.7985,Abu Dhabi Grand Prix,0,1,0.929412,0.860465
4696,2022,2022-11-20,Abu Dhabi Grand Prix,1096,24,852,213,Yuki Tsunoda,AlphaTauri,11,...,0,Japanese,1,2,21.9265,Abu Dhabi Grand Prix,0,0,0.930233,0.921053
4697,2022,2022-11-20,Abu Dhabi Grand Prix,1096,24,854,210,Mick Schumacher,Haas F1 Team,16,...,0,German,11,2,22.9030,Abu Dhabi Grand Prix,0,0,0.939759,0.973684


### Shift position of wins, points, pitstops, pitduration

We do this because we dont want the model to know present information, we only want it to know the information of the race the driver competed in before the present one

In [26]:
final_df['wins'] = final_df.groupby("driverId")['wins'].shift().fillna(0)
final_df['points'] = final_df.groupby('driverId')['points'].shift().fillna(0)

final_df['constructorPoints'] = final_df.groupby(['driverId', 'constructorId'])['constructorPoints'].shift().fillna(0)
final_df['constructorWins'] = final_df.groupby(['driverId', 'constructorId'])['constructorWins'].shift().fillna(0)

final_df['pitStops'] = final_df.groupby('driverId')['pitStops'].shift().fillna(0)
final_df['pitDuration'] = final_df.groupby('driverId')['pitDuration'].shift().fillna(0)

In [27]:
df_with_weather = pd.read_csv("F1Stats_Final_v1.csv")

In [28]:
df_with_weather = df_with_weather.drop_duplicates(subset=['raceId', 'driverId'], keep='first')

In [29]:
df_with_weather.columns

Index(['resultId', 'raceId', 'driverId', 'constructorId', 'grid', 'position',
       'positionOrder', 'points', 'laps', 'time', 'milliseconds_x',
       'fastestLap', 'rank', 'fastestLapTime', 'fastestLapSpeed', 'statusId',
       'year', 'round', 'circuitId', 'raceName', 'date', 'raceTime',
       'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time',
       'quali_date', 'quali_time', 'sprint_date', 'sprint_time', 'name',
       'location', 'country', 'lat', 'lng', 'alt', 'wins', 'code', 'forename',
       'surname', 'nationality', 'constructorName', 'constructorPoints',
       'constructorPosition', 'constructorPositionText', 'constructorWins',
       'stop', 'lap', 'pitTime', 'pitDuration', 'milliseconds_y', 'weather',
       'rain', 'snow', 'cloudy'],
      dtype='object')

In [30]:
final_df.columns

Index(['year', 'date', 'GP_name', 'raceId', 'circuitId', 'driverId',
       'constructorId', 'driver', 'constructor', 'position', 'quali_pos',
       'wins', 'points', 'constructorPoints', 'constructorWins', 'driver_home',
       'statusId', 'pitStops', 'pitDuration', 'GP_name', 'constructor_dnf',
       'driver_dnf', 'constructor_reliability', 'driver_confidence'],
      dtype='object')

In [31]:
len(df_with_weather)

4699

In [32]:
len(final_df)

4699

In [33]:
final_df_weather = final_df.merge(df_with_weather[['raceId', 'driverId', 'weather', 'rain', 'snow', 'cloudy']], on=['raceId', 'driverId'], how='inner')

In [34]:
len(final_df_weather)

4699

In [35]:
final_df.to_csv('F1Stats_noWeather_filtered.csv')
final_df_weather.to_csv('F1Stats_withWeather_filtered.csv')